In [14]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu

In [31]:
def failed_generation_index(dataset: pd.DataFrame):
    return dataset.loc[dataset['has_error'] == True].index

In [39]:
df_test

,row,last_executed_step,to_be_executed_step,translated_prompt,status,has_error,input,target
2418,"""Can you find all cats with pictures and displ...",LLMTranslator,,SELECT ?cat ?catLabel ?image WHERE {\n ?cat w...,,False,"""Can you find all cats with pictures and displ...",SELECT ?item ?itemLabel ?pic\nWHERE\n{\n?item ...
2419,"""Can you retrieve a list of people who are not...",LLMTranslator,,SELECT ?person ?personLabel ?personDescription...,,False,"""Can you retrieve a list of people who are not...",SELECT ?person ?personLabel ?personDescription...
2420,"""Can you write a SparQL query to find the 50 l...",LLMTranslator,,SELECT ?word ?wordLabel WHERE {\n ?word wdt:P...,,False,"""Can you write a SparQL query to find the 50 l...",SELECT ?id WHERE { ?lexeme [entity 0] ?id. } G...
2421,"""Can you find out where the authors of Wikimam...",LLMTranslator,,SELECT ?author ?authorLabel ?birthPlaceLabel W...,,False,"""Can you find out where the authors of Wikimam...",SELECT ?livre ?livreLabel ?personneLabel ?lieu...
2422,"""Can you find out how many online accounts are...",LLMTranslator,,SELECT (COUNT(DISTINCT ?person) AS ?count) WHE...,,False,"""Can you find out how many online accounts are...",SELECT\n?property\n?propertyLabel\n?count\nWIT...


In [38]:
df_ref = pd.read_pickle("../outputs/finetune_dataset_test.pkl")
df_test = pd.read_parquet("../outputs/wikidatallm/results.parquet.gzip")
failed_index = failed_generation_index(df_test)
# df_ref = df_ref.drop(failed_index)
df_test = df_test.drop(failed_index)


In [8]:
print(df_test['translated_prompt'][0])

SELECT COUNT(DISTINCT ?country) WHERE {
?country wdt:P31 wd:Q236. # P31 is "instance of", Q236 is "country"
?country wdt:P279 wd:Q1041. # P279 is "member of", Q1041 is "European Union"
}



In [16]:
corpus_bleu([x.split() for x in df_ref['target']], [x.split() for x in df_test['translated_prompt']])

AttributeError: 'NoneType' object has no attribute 'split'